In [124]:
if 'etl.preprocessing' in sys.modules:  
    print("Replaced")
    del sys.modules["etl.preprocessing"]

from etl.feature_extractor import Items, Interactions
from etl.preprocessing import Preprocessor

Replaced


In [125]:
items = Items()
items.items

In [126]:
interactions = Interactions()

In [127]:
model_data = Preprocessor(items, interactions, 'visitorid','itemid')

In [128]:
model_data.rate_matrix

{'train': <121911x61686 sparse matrix of type '<class 'numpy.int64'>'
 	with 178962 stored elements in COOrdinate format>,
 'test': <121911x61686 sparse matrix of type '<class 'numpy.int64'>'
 	with 12058 stored elements in COOrdinate format>,
 'feature': <61686x1258 sparse matrix of type '<class 'numpy.int64'>'
 	with 132326 stored elements in COOrdinate format>}

In [360]:
model = LightFM(no_components=20, loss='warp', learning_rate=0.01)
model.fit(
    model_data.rate_matrix['train']
    , item_features = None#model_data.rate_matrix['feature']
    , epochs=50, num_threads=8)

In [361]:
auc_without_features = auc_score(model = model, 
                        test_interactions = model_data.rate_matrix['test'],
                        num_threads = 4, check_intersections = False)
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_without_features.mean(), 2))

precision_without_features = precision_at_k(model = model, 
                        test_interactions = model_data.rate_matrix['test'],
                        num_threads = 4, check_intersections = False)
print("average Precision without adding item-feature interaction = {0:.{1}f}".format(precision_without_features.mean(), 2))

recall_without_features = recall_at_k(model = model, 
                        test_interactions = model_data.rate_matrix['test'],
                        num_threads = 4, check_intersections = False)
print("average Recall without adding item-feature interaction = {0:.{1}f}".format(recall_without_features.mean(), 2))

average AUC without adding item-feature interaction = 0.80
average Precision without adding item-feature interaction = 0.01
average Recall without adding item-feature interaction = 0.04


In [362]:
model = LightFM(no_components=20, loss='warp', learning_rate=0.01)
model.fit(
    model_data.rate_matrix['train']
    , item_features = model_data.rate_matrix['feature']
    , epochs=50, num_threads=8)

In [363]:
auc_without_features = auc_score(model = model, 
                        test_interactions = model_data.rate_matrix['test'],
                        item_features = model_data.rate_matrix['feature'],
                        num_threads = 4, check_intersections = False)
print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_without_features.mean(), 2))

precision_without_features = precision_at_k(model = model, 
                        test_interactions = model_data.rate_matrix['test'],
                        item_features = model_data.rate_matrix['feature'],
                        num_threads = 4, check_intersections = False)
print("average Precision without adding item-feature interaction = {0:.{1}f}".format(precision_without_features.mean(), 2))

recall_without_features = recall_at_k(model = model, 
                        test_interactions = model_data.rate_matrix['test'],
                        item_features = model_data.rate_matrix['feature'],
                        num_threads = 4, check_intersections = False)
print("average Recall without adding item-feature interaction = {0:.{1}f}".format(recall_without_features.mean(), 2))

average AUC without adding item-feature interaction = 0.90
average Precision without adding item-feature interaction = 0.00
average Recall without adding item-feature interaction = 0.01


In [213]:
results = model_data.interactions.test[:1]
results['recommemdation'] = model.predict(
    user_ids = model_data.cate_enc_dict['visitorid'].transform(results['visitorid'])
    , item_ids = model_data.cate_enc_dict['itemid'].transform(results['itemid'])
    , item_features = model_data.rate_matrix['feature']
)
results

/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,visitorid,itemid,rating,recommemdation
151453,517086,138017,1,5.283116


In [336]:
def predict_recom(user_id_orig, item_id_orig, model_data, recom_num, model):
    result_dict = dict()
    result_array = list()
    user_id = model_data.cate_enc_dict['visitorid'].transform(user_id_orig)
    item_id = model_data.cate_enc_dict['itemid'].transform(item_id_orig)
    #Known Positives
    known_positives = np.unique(
        np.concatenate
        (
            (model_data.interactions.train['itemid'].values
             [np.where(model_data.trans_cat_train['visitorid']==user_id)]
                    , model_data.interactions.test['itemid'].values[
                        np.where(model_data.trans_cat_test['visitorid']==user_id)]), axis=0))
    
    #Recommended
    scores = np.unique(model.predict(
        user_ids = user_id
    , item_ids = item_id
    , item_features = model_data.rate_matrix['feature']
    )
                      )

    top_items = model_data.trans_cat_test['itemid'][np.argsort(-scores)]
    # printing out the result
    print("User %s" % model_data.cate_enc_dict['visitorid'].inverse_transform([user_id])[0])
    print("     Known positives:")

    for x in known_positives[:recom_num]:
        print("                  %s" % x)


    print("     Recommended:",item_id)
    print(similar_items(
    item_id[0]
    , model_data.rate_matrix['feature']
    , model))
#     for x in top_items[:5]:
#         print("                  %s" % model_data.cate_enc_dict['itemid'].inverse_transform([x])[0])
#         result_array.append(x)
# #     result_dict[user_id_orig] = result_array
    print("")
#     return results_dict

def similar_items(internal_idx, item_feats_mtx, model, N=10):
    item_representations = item_feats_mtx.dot(model.item_embeddings)

    # Cosine similarity
    scores = item_representations.dot(item_representations[internal_idx, :])
    item_norms = np.linalg.norm(item_representations, axis=1)
    scores /= item_norms

    best = np.argpartition(scores, -N)[-N:]
    return sorted(zip(best, scores[best] / item_norms[internal_idx]),
                  key=lambda x: -x[1])

In [342]:
print(model_data.interactions.train[0:2])

top_rec = predict_recom(model_data.interactions.test['visitorid'][1:2]
                        , model_data.interactions.test['itemid'][1:2]
                        , model_data
                        , 5, model)

        visitorid  itemid  rating
0         1155627   61279       1
211645     393706  359990       1
User 247041
     Known positives:
                  178621
                  217712
                  257245
                  405934
     Recommended: [33940]
[(41202, 1.0), (49497, 1.0), (19683, 1.0), (14357, 1.0), (9807, 1.0), (19688, 1.0), (56825, 1.0), (16317, 1.0), (44652, 1.0), (53711, 1.0)]



/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [340]:
def similar_items(internal_idx, item_feats_mtx, model, N=10):
    item_representations = item_feats_mtx.dot(model.item_embeddings)

    # Cosine similarity
    scores = item_representations.dot(item_representations[internal_idx, :])
    item_norms = np.linalg.norm(item_representations, axis=1)
    scores /= item_norms

    best = np.argpartition(scores, -N)[-N:]
    return sorted(zip(best, scores[best] / item_norms[internal_idx]),
                  key=lambda x: -x[1])

In [341]:
similar_items(
    61685
    , model_data.rate_matrix['feature']
    , model)
# model_data.rate_matrix['feature'].shape #(61686, 1258)
# # model.item_embeddings.shape #[1258,10]

# item_representations = model_data.rate_matrix['feature'].dot(model.item_embeddings)
# scores = item_representations.dot(item_representations[61685, :])
# len(scores)

[(50281, 1.0),
 (2233, 1.0),
 (23272, 1.0),
 (14868, 1.0),
 (37582, 1.0),
 (20090, 1.0),
 (22009, 1.0),
 (558, 1.0),
 (49437, 1.0),
 (61685, 1.0)]

In [332]:
txfm_list = model_data.cate_enc_dict['itemid'].transform(model_data.item_list)
txfm_list

array([    0,     1,     2, ..., 61683, 61684, 61685])

In [281]:
model_data.item_list [cosine_similarity(model.user_embeddings, model.item_embeddings).argsort()][::-1].shape

(121911, 1258)

In [292]:
print(model_data.item_list[model.item_biases.argsort()][-10:])

[8426 1879 5758 2036 4019  381 2504 7412 7777 8890]


In [296]:
cosine_similarity(model.user_embeddings, model.item_embeddings).shape

(121911, 1258)